In [ ]:
import pandas as pd
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import random


def scrape_page(page_num):
    try:
        time.sleep(random.uniform(0.1, 0.3))
        url = "https://www.boliga.dk/salg/resultater?searchTab=1&page=%d&sort=date-d" % page_num
        d = pd.read_html(url)[0]
        print("Page %d done" % page_num)
        return d
    except Exception as e:
        print("Page %d failed: %s" % (page_num, str(e)))
        raise e  

start_time = time.time()
dl = []

try:
    with ThreadPoolExecutor(max_workers=30) as executor:
        # Submit all tasks
        futures = [executor.submit(scrape_page, i) for i in range(1, 35699)]
        
        # Collect results as they complete
        for future in as_completed(futures):
            result = future.result()  # This will raise exception if page failed
            dl.append(result)

except Exception as e:
    print(f"Scraping stopped due to error: {e}")

# Concatenate whatever we got before the error
if dl:
    df = pd.concat(dl, ignore_index=True)
    print(f"Successfully scraped {len(dl)} pages before stopping")
else:
    print("No successful pages scraped")

Page 5 done
Page 7 done
Page 10 done
Page 1 done
Page 9 done
Page 3 done
Page 8 done
Page 4 done
Page 2 done
Page 6 done
Page 17 done
Page 13 done
Page 15 done
Page 11 done
Page 12 done
Page 14 done
Page 16 done
Page 18 done
Page 20 done


KeyboardInterrupt: 

Page 19 done
Page 22 done
Page 23 done
Page 27 done
Page 25 done
Page 26 done
Page 21 done
Page 24 done
Page 28 done
Page 29 done
Page 30 done
Page 31 done
Page 33 done
Page 32 done
Page 35 done
Page 36 done
Page 37 done
Page 38 done
Page 34 done
Page 39 done
Page 40 done
Page 41 done
Page 42 done
Page 43 done
Page 44 done
Page 45 done
Page 46 done
Page 47 done
Page 49 done
Page 50 done
Page 48 done
Page 51 done
Page 53 done
Page 52 done
Page 54 done
Page 56 done
Page 55 done
Page 57 done
Page 58 done
Page 59 done
Page 60 done
Page 62 done
Page 65 done
Page 61 done
Page 63 done
Page 64 done
Page 67 done
Page 66 done
Page 69 done
Page 68 done
Page 70 done
Page 73 done
Page 74 done
Page 71 done
Page 75 done
Page 76 done
Page 72 done
Page 77 done
Page 78 done
Page 80 done
Page 79 done
Page 82 done
Page 81 done
Page 84 done
Page 83 done
Page 85 done
Page 87 done
Page 86 done
Page 88 done
Page 89 done
Page 90 done
Page 91 done
Page 92 done
Page 93 done
Page 94 done
Page 95 done
Page 97 done

In [16]:
print(futures[0].result())

                                  Boligtype & Adresse         Købesum  \
0   Fritidshus FFritidshus Duevej 14, Dybendal  41...   1.495.000 kr.   
1       Villa VVilla Grønlandsvej 16  7000 Fredericia   1.020.000 kr.   
2          Villa VVilla Fensmarks Alle 21  3520 Farum   8.370.000 kr.   
3   Ejerlejlighed EEjerlejlighed Lergravsvej 56, 1...   5.062.500 kr.   
4            Rækkehus RRækkehus Elmely 22  3520 Farum   3.675.000 kr.   
5   Ejerlejlighed EEjerlejlighed Broholms Alle 20,...   4.195.000 kr.   
6         Villa VVilla Gefionsvej 19  7000 Fredericia   2.565.000 kr.   
7            Villa VVilla Gasværksvej 24  4300 Holbæk   2.500.000 kr.   
8         Villa VVilla Endelavevej 2  5500 Middelfart   3.025.000 kr.   
9   Ejerlejlighed EEjerlejlighed Kantorparken 12, ...     600.000 kr.   
10  Fritidshus FFritidshus Sleipnersvej 38  3210 V...   2.195.000 kr.   
11  Ejerlejlighed EEjerlejlighed Astilbehaven 137,...   5.700.000 kr.   
12             Villa VVilla Svalevej 4  4100 Ringst

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.to_csv(r'file1.csv', index=False)

In [ ]:
end = time.time()
print("Execution time: ", end - start,"secs")